In [2]:
# Area under curve for finding step frequecny
# Find step frequencies for continuous segments
# Speed find the running speed in continuous intervals

import Data
from IPython.display import clear_output
from tensorflow.keras import layers
from keras import backend as k
import IPython
import numpy as np
import Data
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import model
from model import build_model_CNN
from plotter import hist_plotter
from sklearn.preprocessing import OneHotEncoder

import os

In [3]:
# Get Data Here
Features_A, Labels, pIDs = Data.dataset_main(150,50,'Yes')
Features_G, Labels, pIDs = Data.dataset_main(150,50,'No')
clear_output()
print('Dataset Loaded')

Dataset Loaded


In [4]:
pIDsUnique = np.unique(pIDs)
pIDsUnique
pIDsInts = np.arange(len(pIDsUnique))
pIDsDict = dict()
for i, ID in enumerate(pIDsUnique):
    pIDsDict[ID] = pIDsInts[i]

pIDsVals = np.array([pIDsDict[ID] for ID in pIDs]).reshape(-1,1)

oneHot = OneHotEncoder(sparse=False)
pIDsEnc = oneHot.fit_transform(pIDsVals)

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    print('Could not initialize the tensorflow gpu')
    pass

In [6]:

def R_Square(y_true, y_pred):

    Num = k.sum(k.square(y_true - y_pred))
    Denom = k.sum(k.square(y_true - k.mean(y_true)))
    R = 1 - Num / (Denom + k.epsilon())
    return R


def build_model_CNN():
    inputA = layers.Input(shape=(3, 150, 1))
    modelA = inputA
    min_conv1 = 27
    max_conv1 = 36
    min_conv2 = 36
    max_conv2 = 45
    modelA = layers.Conv2D(27,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(inputA)

    modelA = layers.Conv2D(45,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(modelA)

    #modelA = layers.Flatten()(modelA)
    modelA = layers.GlobalMaxPool2D()(modelA)
    inputG = layers.Input(shape=(3, 150, 1))

    modelG = inputG

    modelG = layers.Conv2D(27,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(inputG)

    modelG = layers.Conv2D(45,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(modelG)

    # model = layers.Dropout(0.4)(model)
    #modelG = layers.Flatten()(modelG)
    modelG = layers.GlobalMaxPool2D()(modelG)
    model = layers.Concatenate()([modelA, modelG])
    #   layers.concatenate(modelA,modelG)

    #model = layers.Dropout(0.4)(model)

    model = layers.Dense(180, activation='relu')(model)

    model = layers.Dropout(0.3)(model)

    model = layers.Dense(30, activation='relu')(model)

    model = layers.Dropout(0.3)(model)

    output = layers.Dense(1)(model)

    model = tf.keras.Model(inputs=[inputA, inputG], outputs=output)

    optimizer1 = tf.keras.optimizers.RMSprop(1e-3)
    # optimizer = tf.keras.optimizers.Adam(1e-3)

    loss1 = tf.keras.losses.MeanAbsoluteError(name="mean_absolute_error")
    
    loss2 = tf.keras.losses.MeanAbsoluteError(name="mean_absolute_error")

    model.compile(
        loss=loss1,  #'mean_absolute_error',
        optimizer=optimizer1,
        metrics=['mae', 'mape', R_Square])

    tf.keras.utils.plot_model(model,
                            #  to_file='/home/redev/Pictures/Model.png',
                              show_shapes=True)
    return model



In [7]:


class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)



earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                 patience=50)

reduceLRplateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                       min_lr=1e-6,
                                                       patience=30,
                                                       factor=0.9,
                                                       verbose=1)

if os.path.isdir('logdir'):
    pass
else:
    os.mkdir('logdir')

logdir = 'logdir'
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)



In [8]:
model = build_model_CNN()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 150, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3, 150, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 3, 150, 27)   270         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 3, 150, 27)   270         input_2[0][0]                    
______________________________________________________________________________________________

In [9]:
import tensorflow.compat.v2 as tf


class EpochDots(tf.keras.callbacks.Callback):
  """A simple callback that prints a "." every epoch, with occasional reports.
  Args:
    report_every: How many epochs between full reports
    dot_every: How many epochs between dots.
  """

  def __init__(self, report_every=100, dot_every=1):
    self.report_every = report_every
    self.dot_every = dot_every

  def on_epoch_end(self, epoch, logs):
    if epoch % self.report_every == 0:
      print()
      print('Epoch: {:d}, '.format(epoch), end='')
      for name, value in sorted(logs.items()):
        print('{}:{:0.4f}'.format(name, value), end=',  ')
      print()

    if epoch % self.dot_every == 0:
      print('.', end='')

In [10]:
FeatureAcol = [feature.tostring() for feature in Features_A]
FeatureGcol = [feature.tostring() for feature in Features_G]
LabelsCol = Labels.ravel()
pIDsCol = pIDsVals.ravel()

dataDF = pd.DataFrame(list(zip(FeatureAcol, FeatureGcol, LabelsCol, pIDsCol)), columns = ['FeatA', 'FeatG', 'Speed', 'ID'])

In [11]:
groupDF = dataDF.groupby(['ID', 'Speed'])

In [23]:
def get_numpyArray(arr):
    return(np.array([np.frombuffer(feat).reshape(3,150,1) for feat in arr]))


def dataParser(dataDF, id, speed):
    print(dataDF.shape)
    
    TrainDF = dataDF[(dataDF['ID']!=id)&(dataDF['Speed']!=speed)]
    TestDF = dataDF[(dataDF['ID']==id)&(dataDF['Speed']==speed)]
    print(TrainDF.shape)
    print(TestDF.shape)
        
    X_TrainA = get_numpyArray(TrainDF['FeatA'])
    X_TrainG = get_numpyArray(TrainDF['FeatG'])
    y_TrainSpeed = TrainDF['Speed'].to_numpy().reshape(-1,1)
    y_TrainID = TrainDF['ID'].to_numpy()
    
    X_TestA = get_numpyArray(TestDF['FeatA'])
    X_TestG = get_numpyArray(TestDF['FeatG'])
    y_TestSpeed = TestDF['Speed'].to_numpy().reshape(-1,1)
    y_TestID = TestDF['ID'].to_numpy()
    
    print(y_TrainSpeed.shape)
    print(y_TrainSpeed.shape)
    print(X_TrainA.shape)
    print(X_TrainG.shape)
        
    X_TrainA, X_ValA, y_TrainA, y_ValSpeed = train_test_split(X_TrainA, y_TrainSpeed, test_size=0.2, shuffle=True, random_state=90)
    X_TrainG, X_ValG, y_TrainG, y_ValSpeed = train_test_split(X_TrainG, y_TrainSpeed, test_size=0.2, shuffle=True, random_state=90)
    
    return(X_TrainA, X_TrainG, y_TrainA, X_TestA, X_TestG, y_TestSpeed, X_ValA, X_ValG,  y_ValSpeed)

In [43]:
results_df = pd.DataFrame(columns = ['id', 'speed','loss', 'mae', 'mape', 'r2','me'])
from keras import backend as K
for id, speed in groupDF.groups:    
    X_TrainA, X_TrainG, y_Train, X_TestA, X_TestG, y_Test, X_ValA, X_ValG, y_Val = dataParser(dataDF, id,speed)

    hist = model.fit(
    [X_TrainA, X_TrainG],
    y_Train,
 #   validation_data=([Features_A, Features_G], Labels),
   validation_data=([X_ValA,X_ValG],y_Val),
        epochs=200,
    verbose=0,
    callbacks=[
        
        EpochDots(report_every=50),
    #    reduceLRplateau,
        ClearTrainingOutput()
    ])

    results = [id, speed]
    results.extend(model.evaluate([X_TestA, X_TestG], np.array(y_Test)))
    results.append(model.predict([X_TestA, X_TestG]).mean())
    
    results_df.loc[len(results_df)] = results
    results=list()
    K.clear_session()


1/1 [==============================] - 0s 736us/step - loss: 0.5097 - mae: 0.5097 - mape: 8.4953 - R_Square: -95771384.0000


In [45]:
results_df['difference'] = results_df['speed']- results_df['me']

In [51]:
results_df['mae'].mean()

0.32078277441055586

In [35]:

model.evaluate([X_TestA, X_TestG], np.array(y_Test))

1/1 [==============================] - 0s 559us/step - loss: 0.2344 - mae: 0.2344 - mape: 7.3238 - R_Square: -16949208.0000


[0.2343616634607315, 0.2343616634607315, 7.323801040649414, -16949208.0]

In [39]:
from sklearn.metrics import r2_score

In [41]:
r2_score(model.predict([X_TestA, X_TestG]), np.array(y_Test).ravel())

-4.267229681802918

In [55]:
results_df.drop(columns ='r2').to_csv('leave_one_speed_out.csv')

In [54]:
results_df.head()

,id,speed,loss,mae,mape,r2,me,difference
0,0.0,3.0,0.571972,0.571972,19.065723,-90419904.0,3.571972,-0.571972
1,0.0,3.2,0.219821,0.219821,6.869399,-16393842.0,3.419821,-0.219821
2,0.0,3.5,0.231096,0.231096,6.602746,-21935340.0,3.573243,-0.073243
3,0.0,4.0,0.288492,0.288492,7.212306,-23341392.0,3.770248,0.229752
4,0.0,4.2,0.510772,0.510772,12.161246,-71306488.0,3.725663,0.474337
